In [108]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

death_table = pd.read_csv('UK Inference Data/out.csv', sep=',').transpose().drop('Unnamed: 0')
death_table=death_table.reset_index()
death_table=death_table.rename(columns={'index':'Name'})
#deaths are accumulated
cumulativedeaths=death_table
for i in range(1,16):
    cumulativedeaths[i]=cumulativedeaths[i-1]+cumulativedeaths[i]

AgeRates = pd.read_csv('UK Inference Data/AgeRate.csv')
AgeRates=AgeRates.rename(columns={'AreaCode':'Name'})
AgeRates=AgeRates.drop(columns=["Unnamed: 0"])


#population data is loaded

resident_data_20182_raw = pd.read_excel('UK Inference Data/ukmidyearestimates20182019ladcodes.xls', 'MYE2-All', skiprows=4)
resident_data_20182_raw=resident_data_20182_raw.drop(columns=['Geography1',"Name"])
resident_data_20182_raw=resident_data_20182_raw.rename(columns={90:'90+'})
resident_data_20182_raw=resident_data_20182_raw.rename(columns={'Code':'AreaCode','All ages':'All Ages'})
resident_data_20182_raw =resident_data_20182_raw[resident_data_20182_raw['AreaCode'].isin(death_table.Name)]
resident_data_2018_raw=resident_data_20182_raw


    
# # We need to age-group the data to match the census workplace data
resident_data_2018 = resident_data_2018_raw[['AreaCode', 'All Ages']]
age_groups = [16, 25, 35, 50, 65, 75]
i = 0
for i_end in age_groups:
    label = '{}-'.format(i)
    age_sum = resident_data_2018_raw[i].copy()
    i += 1
    while i < i_end:
        age_sum += resident_data_2018_raw[i]
        i += 1

    label = label + '{}'.format(i-1)
    resident_data_2018[label] = age_sum

age_sum = resident_data_2018_raw[i].copy()
i += 1
while i < 90:
    age_sum += resident_data_2018_raw[i]
    i += 1
age_sum += resident_data_2018_raw['90+']
resident_data_2018['75+'] = age_sum
resident_data_2018=resident_data_2018.rename(columns={'AreaCode':'Name'})
resident_data_2018=resident_data_2018.reset_index(drop=True)
resident_data_2018=resident_data_2018.merge(cumulativedeaths, on='Name', how='right')
AgeRates=AgeRates.merge(cumulativedeaths, on='Name')

names=death_table.Name

cumulativedeaths=cumulativedeaths.drop(columns=['Name'])

hh= np.array([0.05004547, 0.11524135, 0.16268827, 0.21877324, 0.28080714, 0.33441327, 0.37462764])
cc= np.array([0.09504815, 0.19134113, 0.25551887, 0.32758932, 0.40389807, 0.46764803, 0.51438423])
mm= np.array([0.02029352, 0.08723728, 0.15941184, 0.26756575, 0.4139815,  0.56187667, 0.68527581])
alpha = np.array([0.56762858, 0.44317647, 0.36356883, 0.28385665, 0.21301662, 0.16467453,  0.1350941 ])

# resident_data_20182_raw =resident_data_20182_raw[resident_data_20182_raw['AreaCode'].isin(GB.AreaCode)]
T1 = 12 
T2 = 16
M=7

/home/jakub/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jakub/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [109]:
bound1=int(len(resident_data_2018)/5)
bound2=int(len(resident_data_2018)/5*2)
bound3=int(len(resident_data_2018)/5*3)
bound4=int(len(resident_data_2018)/5*4)

In [110]:
Sf=np.zeros((len(resident_data_2018),M,T2-T1))
Ef=np.zeros((len(resident_data_2018),M,T2-T1))
Af=np.zeros((len(resident_data_2018),M,T2-T1))
Iaf=np.zeros((len(resident_data_2018),M,T2-T1))
Isf=np.zeros((len(resident_data_2018),M,T2-T1))
Imf=np.zeros((len(resident_data_2018),M,T2-T1))
Rf=np.zeros((len(resident_data_2018),M,T2-T1))

In [111]:
for i in range(0, len(resident_data_2018)):
    k=resident_data_2018[resident_data_2018['Name']==death_table['Name'][i]].index.values[0]
    Ni=np.zeros((M))
    Ni[0] = resident_data_2018['0-15'][k].astype('int')
    Ni[1] = resident_data_2018['16-24'][k].astype('int')
    Ni[2] = resident_data_2018['25-34'][k].astype('int')
    Ni[3] = resident_data_2018['35-49'][k].astype('int')
    Ni[4] = resident_data_2018['50-64'][k].astype('int')
    Ni[5] = resident_data_2018['65-74'][k].astype('int')
    Ni[6] = resident_data_2018['75+'][k].astype('int')

    N = np.sum(Ni)
    cumulativedeathsArr=cumulativedeaths.iloc[i].values
    cumulativedeathsArr = np.array(cumulativedeathsArr[T1:T2], dtype='float')
    cumulativedeathsArrDivdM=np.zeros((M,T2-T1))
    totInalphaMM=0
    for j in range(0, M):
        totInalphaMM=mm[j]*cc[j]*hh[j]*(1-alpha[j])*Ni[j]+totInalphaMM

    for j in range(0,M):
        cumulativedeathsArrDivdM[j]=cumulativedeathsArr*mm[j]*cc[j]*hh[j]*(1-alpha[j])*Ni[j]/totInalphaMM/Ni[j]
    Imf[i]=cumulativedeathsArrDivdM

In [90]:
name="data"
sfName="/SF.npy"
efName="/EF.npy"
afName="/AF.npy"
iafName="/IaF.npy"
isfName="/IsF.npy"
rfName="/Rf.npy"

n=name+str(0)+sfname
TempSf=np.load(n)
bounds=np.array([0,bound1, bound2, bound3, bound4,int(len(resident_data_2018))])
for i in range(0, 5):
    nsf=name+str(i)+sfName
    TempS=np.load(nsf)
    nef=name+str(i)+efName
    TempE=np.load(nef)
    naf=name+str(i)+afName
    TempA=np.load(naf)
    niaf=name+str(i)+iafName
    TempIa=np.load(niaf)
    nisf=name+str(i)+isfName
    TempIs=np.load(nisf)
    nrf=name+str(i)+rfName
    TempR=np.load(nrf)
    Sf[bounds[i]:bounds[i+1]]=TempS[bounds[i]:bounds[i+1]]
    Ef[bounds[i]:bounds[i+1]]=TempE[bounds[i]:bounds[i+1]]
    Af[bounds[i]:bounds[i+1]]=TempA[bounds[i]:bounds[i+1]]
    Iaf[bounds[i]:bounds[i+1]]=TempIa[bounds[i]:bounds[i+1]]
    Isf[bounds[i]:bounds[i+1]]=TempIs[bounds[i]:bounds[i+1]]
    Rf[bounds[i]:bounds[i+1]]=TempR[bounds[i]:bounds[i+1]]

In [112]:
name="data"
sfName="/SF.npy"
efName="/EF.npy"
afName="/AF.npy"
iafName="/IaF.npy"
isfName="/IsF.npy"
rfName="/Rf.npy"

Sf=np.load("AllData/SF.npy")
Ef=np.load('AllData/EF.npy')
Af=np.load('AllData/AF.npy')
Iaf=np.load('AllData/IaF.npy')
Isf=np.load('AllData/IsF.npy')
Rf=np.load('AllData/Rf.npy')

In [114]:
for i in range(0,len(Rf)):
    for j in range(0,M):
        if(Rf[i][j][0]<0):
            a=-Rf[i][j][0]
            Rf[i][j]=Rf[i][j]-Rf[i][j][0]
            Sf[i][j]=Sf[i][j]/(1+a)
            Ef[i][j]=Ef[i][j]/(1+a)
            Af[i][j]=Af[i][j]/(1+a)
            Iaf[i][j]=Iaf[i][j]/(1+a)
            Isf[i][j]=Isf[i][j]/(1+a)
            Imf[i][j]=Imf[i][j]/(1+a)

In [116]:
work_age_groups = ['0-15', '16-24', '25-34', '35-49', '50-64', '65-74', '75+']
l=2
for age in range(0, len(work_age_groups)):
    CurrentState= pd.DataFrame(columns=[])
    CurrentState['Name']=names
    CurrentState['S']=pd.Series(Sf[:,age,l])
    CurrentState['E']=pd.Series(Ef[:,age,l])
    CurrentState['A']=pd.Series(Af[:,age,l])
    CurrentState['Ia']=pd.Series(Iaf[:,age,l])
    CurrentState['Is']=pd.Series(Isf[:,age,l])
    CurrentState['R']=pd.Series(Rf[:,age,l])
    CurrentState['Im']=pd.Series(Imf[:,age,l])
    CsvName='CurrentState'+work_age_groups[age]+'.csv'
    CurrentState.to_csv(CsvName)

In [118]:
Sf

array([[[0.99665583, 0.99568032, 0.99426828, 0.99232063],
        [0.99376232, 0.99136018, 0.98794878, 0.98337568],
        [0.99502232, 0.99241706, 0.98826601, 0.9821616 ],
        ...,
        [0.99630683, 0.99095794, 0.98195244, 0.96828658],
        [0.99397324, 0.99212097, 0.98915986, 0.98478693],
        [0.9925516 , 0.99221123, 0.99173956, 0.9911118 ]],

       [[0.99702599, 0.99396295, 0.98700331, 0.97165823],
        [0.99273703, 0.98556389, 0.96982585, 0.93621028],
        [0.98998256, 0.98025145, 0.95780528, 0.90927928],
        ...,
        [0.98597505, 0.9642158 , 0.91393073, 0.80991072],
        [0.98591108, 0.97835235, 0.9616153 , 0.92597825],
        [0.98589894, 0.98461327, 0.98215203, 0.97720097]],

       [[0.99758174, 0.99670258, 0.99528266, 0.99304023],
        [0.99542078, 0.99324342, 0.98981499, 0.98457974],
        [0.99632678, 0.99395912, 0.98972307, 0.9826164 ],
        ...,
        [0.99699242, 0.99218172, 0.98299057, 0.96705646],
        [0.99640369, 0.994811